# Configure Data Node for CMIP6 publishing and access

Publishing
----------
Edit `/esg/config/esgcet/esg.ini` -- global publishing settings.

This file by default must be modified by the node adminstrator, given default permission.

Changes under `DEFAULT` group:

In [ ]:
[DEFAULT]
# Set up Globus endpoint if publishing to Globus - add Globus entry and replace UUID
thredds_file_services =
        HTTPServer | /thredds/fileServer/       | HTTPServer    | fileservice
        GridFTP    | gsiftp://my.esgf.node:2811/ | GRIDFTP       | fileservice
        OpenDAP    | /thredds/dodsC/            | OpenDAPServer | fileservice
        Globus     | globus:UUID/ | Globus | fileservice

#  Add additional THREDDS root, include path up to but not including CMIP6 CMIP6
thredds_dataset_roots =
        esg_dataroot | /esg/data
        my_cmip6_dataroot | /path/to/cmip6/data

# certs directory is commonly missing in old deployments - check and add this
hessian_service_certs_location = /etc/grid-security/certificates
        
#   To see info during publishing check log level
log_level = INFO

Add to `esg.ini` section`config.cmip6` with the following:   

**NOTE:** For `pid_credentials` Nodes in _North America_ should list `pcmdi10.llnl.gov` first

_Europe/Asia_: keep `handle-esgf-trusted.dkrz.de` first in server listing

[config:cmip6]
pid_credentials =
    handle-esgf-trusted.dkrz.de | 5671  | esgf-pid | esgf-publisher | <secret>  | true
    pcmdi10.llnl.gov            | 5671  | esgf-pid | esgf-publisher | <secret>            | true
pid_exchange_name = esgffed-exchange
pid_prefix = 21.14100
cmor_table_path = /usr/local/cmip6-cmor-tables/Tables
data_specs_version = file  # “file” means the publisher will follow the DR tag specified in each file with “data_specs_version” attribut
# FOR test publishing use this one CDF2CIM_CLIENT_WS_HOST = https://test-cdf2cim-api.es-doc.org
CDF2CIM_CLIENT_WS_HOST=https://cdf2cim.es-doc.org  # for production 
CDF2CIM_CLIENT_GITHUB_USER = <your_GitHub_username>
CDF2CIM_CLIENT_GITHUB_ACCESS_TOKEN = <your_GitHub_personal_access_token_for_cdf2cim>


Set up regular user account for cmor table access.  Recommended not to publish from the root account. 

In [ ]:
usermod <linux-username> -G esgfpub -a

Correct thredds catalog permissions 

In [ ]:
chgrp -R esgfpub /esg/content/thredds/esgcet
chmod g+rwx /esg/content/thredds/esgcet
chmod g+rwx /esg/content/thredds/esgcet/catalog.xml

Access
------
Edit `/esg/config/esgf_policies_local.xml`.  Insert a single rule to make data available  freely to all.

In [ ]:
<policies xmlns="http://www.esgf.org/security">
<!--Makes data available to all-->
    <policy resource=".*CMIP6.*" attribute_type="ANY" attribute_value="" action="Read"/>
</policies>

Ensure that you have an up-to-date IdP whitelist file installed (for ESGF versions 2.x; future installations using Ansible should not require this step.

In [ ]:
cd /tmp # or other working directory
git clone https://github.com/ESGF/config
cp config/esgf-prod/esgf_idp_static.xml /esg/config